In [1]:
!git clone https://github.com/speckean/upar_dataset.git

Cloning into 'upar_dataset'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 16 (delta 1), reused 6 (delta 0), pack-reused 0
Unpacking objects: 100% (16/16), 4.87 MiB | 4.94 MiB/s, done.


# Import Tools

In [2]:
import os
import pickle
import glob
import random

import numpy as np
import torch.utils.data as data
from PIL import Image

import torchvision.transforms as T
from torchvision import models

from torch.utils.data import DataLoader

import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
HOME = os.getcwd()
HOME

'/kaggle/working'

# Dataset

## Get a data

In [4]:
!git clone https://github.com/speckean/upar_challenge.git
%cd upar_challenge
!pip install tqdm gdown requests

Cloning into 'upar_challenge'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 49 (delta 14), reused 40 (delta 10), pack-reused 0
Unpacking objects: 100% (49/49), 144.37 KiB | 3.21 MiB/s, done.
/kaggle/working/upar_challenge


In [5]:
!python3 download_datasets.py
%cd {HOME}
!mv upar_challenge/data .

Download Market 1501 dataset
Downloading...
From (original): https://drive.google.com/uc?id=0B8-rUzbwVRk0c054eEozWG9COHM
From (redirected): https://drive.google.com/uc?id=0B8-rUzbwVRk0c054eEozWG9COHM&confirm=t&uuid=321826f4-5fef-4dab-9ec9-196a91433fff
To: /kaggle/working/upar_challenge/data/market_1501.zip
100%|█████████████████████████████████████████| 153M/153M [00:00<00:00, 252MB/s]
Extract Market 1501 dataset
Extracting : 100%|██████████████████████| 68042/68042 [00:07<00:00, 8506.82it/s]
Download PA100k dataset
Retrieving folder contents
Processing file 1dt5nteusNfCVrjAkffLK1sbh4hTLM8tl annotation.zip
Processing file 1Bod3MrbdCRiSvpR7NSOYi-rF3-ushwtj data.zip
Processing file 1zmQkoN2YJgbclT990ihj_DYU29YAidey README.txt
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1dt5nteusNfCVrjAkffLK1sbh4hTLM8tl
To: /kaggle/working/upar_challenge/data/PA100k/annotation.zip
100%|████████

In [6]:
current_dir_path = 'data/PA100k/release_data/release_data'
new_dir_path = 'data/PA100k/data'


if os.path.exists(current_dir_path):
    if not os.path.exists(new_dir_path):
        os.rename(current_dir_path, new_dir_path)
        print(f"Directory renamed from '{current_dir_path}' to '{new_dir_path}'")

    else:
        print(f"The target directory '{new_dir_path}' already exists.")
else:
    print(f"The directory '{current_dir_path}' does not exist.")

Directory renamed from 'data/PA100k/release_data/release_data' to 'data/PA100k/data'


## Construct dataset/dataloader

In [7]:

class UPAR(data.Dataset):
  """
  Load UPAR dataset from pickle file

  split: whether to use train/val/trainval/test split
  partition: partition id 0-9
  root: path to datasets, original datasets must be in this directory
  data_path: path to UPAR pickle file
  transform: training data transforms
  target_transforms: evaluation data transforms
  """
  def __init__(self, split='train', partition=0, root=HOME, data_path='/kaggle/working/upar_dataset/UPAR/dataset_all.pkl', transform=None, target_transform=None):
    dataset_info = pickle.load(open(data_path, 'rb+'))
    self.dataset_info = dataset_info

    img_id = dataset_info.image_name
    attr_label = dataset_info.label

    assert split in dataset_info.partition.keys(), f'split {split} does not exist'

    self.dataset = 'UPAR'
    self.transform = transform  # data transforms during training
    self.target_transform = target_transform  # data transforms during testing

    self.root_path = root+"/data"  # path to datasets

    self.attr_id = dataset_info.attr_name  # attribute names
    self.attr_num = len(self.attr_id)  # number of attributes

		# load partition
    self.img_idx = dataset_info.partition[split]

    if isinstance(self.img_idx, list):
      self.img_idx = self.img_idx[partition]

    if isinstance(self.img_idx, list):
      self.img_idx = np.hstack(self.img_idx)

    self.img_idx = np.array([i for i in self.img_idx if not any(folder in img_id[i] for folder in ["RAP"])])


    self.img_num = self.img_idx.shape[0]
    self.img_id = [img_id[i] for i in self.img_idx]
    self.label = attr_label[self.img_idx]

    # set sub-dataset lengths to enable evaluation on sub-datasets
    self.sub_dataset_lengths = [len(d) for d in dataset_info.partition.test[partition]]


  def __getitem__(self, index):
      """
      get dataset item by index

      index: item index
      return: image data (img) with corresponding ground truth labels (gt_label), dataset id (did), and image path (imgname)
      """
      imgname, gt_label, imgidx = self.img_id[index], self.label[index], self.img_idx[index]
      did = self.dataset_info.dataset_ids[imgidx]
      imgpath = os.path.join(self.root_path, imgname)
      img = Image.open(imgpath)

      if self.transform is not None:
          img = self.transform(img)

      gt_label = gt_label.astype(np.float32)

      if self.target_transform is not None:
          gt_label = self.transform(gt_label)

      return img, gt_label, did, imgname


  def __len__(self):
    """
    get length of dataset
    """
    return len(self.img_id)


def get_transform(height, width):
    normalize = T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    train_transform = [
        T.Resize((height, width))
    ]

    train_transform += [
        T.Pad(10),
        T.RandomCrop((height, width)),
        T.RandomHorizontalFlip(),
    ]

    train_transform += [
        T.ToTensor(),
        normalize,
    ]
    train_transform = T.Compose(train_transform)

    valid_transform = T.Compose([
        T.Resize((height, width)),
        T.ToTensor(),
        normalize
    ])

    return train_transform, valid_transform

In [8]:
height = 224
width = 224
train_transform, valid_transform = get_transform(height, width)


train_dataset = UPAR(split='train', partition=0, transform=train_transform)
val_dataset = UPAR(split='val', partition=0, transform=valid_transform)
test_dataset = UPAR(split='test', partition=0, transform=valid_transform)


In [9]:
batch_size = 32 
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Model Baseline

In [10]:
model = models.mobilenet_v2(pretrained=True)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 99.4MB/s]


In [11]:
num_classes = 40
model.classifier[1] = nn.Linear(model.last_channel, num_classes)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [12]:
def train_one_epoch(model, dataloader, loss_fn, optimizer, device):
    model.train()
    total_loss = 0
    for batch, (X, y, _, _) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        if batch % 100 == 0:
            print(f"Batch {batch}, Loss: {loss.item()}")

    avg_loss = total_loss / len(dataloader)
    print(f"Average Training Loss: {avg_loss}")
    return avg_loss

def validate(model, dataloader, loss_fn, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for X, y, _, _ in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            loss = loss_fn(pred, y)
            total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f"Average Validation Loss: {avg_loss}")
    return avg_loss

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

epochs = 5

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    print("-" * 10)
    train_loss = train_one_epoch(model, train_loader, loss_fn, optimizer, device)
    val_loss = validate(model, val_loader, loss_fn, device)


Epoch 1/5
----------
Batch 0, Loss: 25.45807647705078
Batch 100, Loss: 18.39025115966797
Batch 200, Loss: 18.3584041595459
Batch 300, Loss: 18.63368034362793
Batch 400, Loss: 18.158985137939453
Batch 500, Loss: 17.447589874267578
Batch 600, Loss: 17.917827606201172
Batch 700, Loss: 17.8455753326416
Batch 800, Loss: 18.20815658569336
Batch 900, Loss: 17.280881881713867
Batch 1000, Loss: 16.63712501525879
Batch 1100, Loss: 17.065988540649414
Batch 1200, Loss: 16.85506820678711
Batch 1300, Loss: 17.157058715820312
Batch 1400, Loss: 17.748868942260742
Batch 1500, Loss: 16.164506912231445
Batch 1600, Loss: 16.10417938232422
Batch 1700, Loss: 16.47708511352539
Batch 1800, Loss: 16.555679321289062
Batch 1900, Loss: 18.177427291870117
Batch 2000, Loss: 16.640670776367188
Batch 2100, Loss: 17.865280151367188
Batch 2200, Loss: 16.705068588256836
Batch 2300, Loss: 15.554530143737793
Batch 2400, Loss: 16.983123779296875
Batch 2500, Loss: 16.99689292907715
Batch 2600, Loss: 16.80962562561035
Batch 

In [13]:
torch.save(model.state_dict(), "mobilenet_model.pth")